In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# --- Crop Recommendation System ---
crop_df = pd.read_csv("crop_recommendation.csv")

crop_dict = {
    'rice': 1, 'maize': 2, 'jute': 3, 'cotton': 4, 'coconut': 5, 'papaya': 6,
    'orange': 7, 'apple': 8, 'muskmelon': 9, 'watermelon': 10, 'grapes': 11,
    'mango': 12, 'banana': 13, 'pomegranate': 14, 'lentil': 15, 'blackgram': 16,
    'mungbean': 17, 'mothbeans': 18, 'pigeonpeas': 19, 'kidneybeans': 20,
    'chickpea': 21, 'coffee': 22
}
reverse_crop_dict = {v: k for k, v in crop_dict.items()}
crop_df['label'] = crop_df['label'].map(crop_dict)

X_crop = crop_df.drop('label', axis=1)
y_crop = crop_df['label']
X_crop_train, X_crop_test, y_crop_train, y_crop_test = train_test_split(X_crop, y_crop, test_size=0.2, random_state=42)

mx = MinMaxScaler()
sc = StandardScaler()
X_crop_train_scaled = sc.fit_transform(mx.fit_transform(X_crop_train))
X_crop_test_scaled = sc.transform(mx.transform(X_crop_test))

crop_model = RandomForestClassifier(random_state=42)
crop_model.fit(X_crop_train_scaled, y_crop_train)

with open("crop_model.pkl", "wb") as f:
    pickle.dump((crop_model, mx, sc, reverse_crop_dict), f)

# --- Fertilizer Recommendation System ---
fert_df = pd.read_csv("fertilizers_recommendation.csv")
fert_df.columns = fert_df.columns.str.strip()

label_encoders = {}
categorical_cols = ['Soil Type', 'Crop Type', 'Fertilizer Name']
for col in categorical_cols:
    le = LabelEncoder()
    fert_df[col] = le.fit_transform(fert_df[col])
    label_encoders[col] = le

X_fert = fert_df.drop("Fertilizer Name", axis=1)
y_fert = fert_df["Fertilizer Name"]
X_fert_train, X_fert_test, y_fert_train, y_fert_test = train_test_split(X_fert, y_fert, test_size=0.2, random_state=42)

fert_model = RandomForestClassifier(random_state=42)
fert_model.fit(X_fert_train, y_fert_train)

with open("fertilizer_model.pkl", "wb") as f:
    pickle.dump((fert_model, label_encoders), f)

# --- Crop to Fertilizer Mapping ---
crop_to_fert_crop_map = {
    'Rice': 'Paddy',
    'Maize': 'Maize',
    'Cotton': 'Cotton',
    'Banana': 'Maize',
    'Mango': 'Maize',
    'Grapes': 'Maize',
    'Apple': 'Maize',
    'Coconut': 'Paddy',

}

# --- Prediction Functions ---
def recommend_crop(N, P, K, temperature, humidity, ph, rainfall):
    features = np.array([[N, P, K, temperature, humidity, ph, rainfall]])
    features_scaled = sc.transform(mx.transform(features))
    crop_id = crop_model.predict(features_scaled).item()
    return reverse_crop_dict.get(crop_id, "Unknown Crop")

def recommend_fertilizer(input_data):
    data = input_data.copy()

    crop_type = data['Crop Type'].capitalize()
    known_crop_types = label_encoders['Crop Type'].classes_

    fert_crop_type = crop_to_fert_crop_map.get(crop_type, None)
    if fert_crop_type is None:
        print(f"⚠️ No fertilizer mapping available for crop '{crop_type}'. Returning default fertilizer.")
        return "Generic Fertilizer Recommendation"
    crop_type = fert_crop_type

    if crop_type not in known_crop_types:
        print(f"⚠️ Fertilizer dataset does not contain crop '{crop_type}'. Returning default fertilizer.")
        return "Generic Fertilizer Recommendation"

    try:
        data['Soil Type'] = label_encoders['Soil Type'].transform([data['Soil Type']])[0]
    except ValueError:
        print(f"⚠️ Unknown Soil Type '{data['Soil Type']}'. Returning default fertilizer.")
        return "Generic Fertilizer Recommendation"

    data['Crop Type'] = label_encoders['Crop Type'].transform([crop_type])[0]

    ordered_cols = ['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous']
    input_df = pd.DataFrame([[data[col] for col in ordered_cols]], columns=ordered_cols)


    pred_encoded = fert_model.predict(input_df)[0]
    return label_encoders['Fertilizer Name'].inverse_transform([pred_encoded])[0]

# --- User Console Input ---
def run_recommendation():
    print("\n📥 Enter soil and climate parameters:")
    N = float(input("Nitrogen (N): "))
    P = float(input("Phosphorus (P): "))
    K = float(input("Potassium (K): "))
    temperature = float(input("Temperature (°C): "))
    humidity = float(input("Humidity (%): "))
    ph = float(input("pH: "))
    rainfall = float(input("Rainfall (mm): "))

    crop = recommend_crop(N, P, K, temperature, humidity, ph, rainfall)
    print(f"\n🌱 Recommended Crop: {crop}")

    print("\n📥 Enter additional inputs for fertilizer recommendation:")
    soil_type = input("Soil Type (e.g., Sandy, Loamy): ")
    crop_type = crop.capitalize()
    temp = temperature
    hum = humidity
    moist = float(input("Moisture (%): "))

    fert_input = {
    'Temparature': temp,
    'Humidity': hum,
    'Moisture': moist,
    'Soil Type': soil_type,
    'Crop Type': crop_type,
    'Nitrogen': N,
    'Potassium': K,
    'Phosphorous': P
    }


    fertilizer = recommend_fertilizer(fert_input)
    print(f"\n🧪 Recommended Fertilizer: {fertilizer}")

if __name__ == "__main__":
    run_recommendation()


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_crop_pred = crop_model.predict(X_crop_test_scaled)
crop_accuracy = accuracy_score(y_crop_test, y_crop_pred)
print("🌾 Crop Model Accuracy:", crop_accuracy)
print(classification_report(y_crop_test, y_crop_pred, target_names=list(reverse_crop_dict.values())))


In [ ]:
y_fert_pred = fert_model.predict(X_fert_test)
fert_accuracy = accuracy_score(y_fert_test, y_fert_pred)
print("🧪 Fertilizer Model Accuracy:", fert_accuracy)

fert_labels = sorted(np.unique(np.concatenate((y_fert_test, y_fert_pred))))
fert_names = label_encoders['Fertilizer Name'].inverse_transform(fert_labels)

print(classification_report(
    y_fert_test,
    y_fert_pred,
    labels=fert_labels,
    target_names=fert_names
))

